In [1]:
import numpy as np
import cv2
import tensorflow as tf
from IPython.display import Audio 
from IPython.core.display import display

In [2]:
import time

In [3]:
import random
from pathlib import Path
import os
import eyed3

In [4]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [5]:
model = tf.keras.models.load_model("6")

In [6]:
def prepare(ima):
    IMG_SIZE = 48        # image size
    img_array = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    img_array=img_array/255.0  
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,1)

In [7]:
EMOTIONS = ["afraid","angry","disgust","happy","neutral","sad","surprised"]

In [8]:
emotion = {
    '0' : 'Afraid',
    '1' : 'Angry',
    '2' : 'Disgust',
    '3' : 'Happy',
    '4' : 'Neutral',
    '5' : 'Sad',
    '6' : 'Surprise'
}

In [34]:
def videocap():
    cap=cv2.VideoCapture(0)
    emotion_count = [0,0,0,0,0,0,0]
    start = time.time()
    while (time.time()-start < 10):
        ret, img=cap.read()

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 8)
        canvas = np.zeros((256,256,3), dtype="uint8")
        frameclone=img
        try:
            faces = sorted(faces, reverse=True, key = lambda x: (x[2]-x[0]) *(x[3]-x[1]))[0]
            (x,y,w,h)=faces
            img = cv2.rectangle(img,(x,y),(x+w,y+h),(220,40,50),2)
            roi = img[y:y+h, x:x+w]

            prediction = (model.predict([prepare(roi)]))
            preds = prediction[0]
            label = EMOTIONS[preds.argmax()]
            emotion_count[preds.argmax()]+=1
            
            cv2.rectangle(img,(x,y+h+10),(x+w,y+h+70),(220,40,50),-2)
            cv2.putText(img,label, (x+10, y+h+50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (225, 225, 225), 3)
            #cv2.putText(img,label, (x+w, y+h-40), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (220, 40, 50), 3)
            #result.write(img)
        except:
            pass
        cv2.imshow('img',img)

        if cv2.waitKey(1) & cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    
    emo = emotion[str(np.argmax(emotion_count))]
    
    return emo

In [ ]:
#EMOTIONS = ["afraid","angry","disgust","happy","neutral","sad","surprised"]

In [ ]:
#emotion = {
#    '0' : 'Afraid',
#    '1' : 'Angry',
#    '2' : 'Disgust',
#    '3' : 'Happy',
#    '4' : 'Neutral',
#    '5' : 'Sad',
#    '6' : 'Surprise'
#}

In [ ]:
#def videoemotion(directory):
#    cap=cv2.VideoCapture(directory)
#
#    while cap.isOpened():
#        ret, img=cap.read()
#        if not ret:
#            break
#
#        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#        faces = face_cascade.detectMultiScale(gray, 1.3, 8)
#        canvas = np.zeros((256,256,3), dtype="uint8")
#        frameclone=img
#        try:
#            faces = sorted(faces, reverse=True, key = lambda x: (x[2]-x[0]) *(x[3]-x[1]))[0]
#            (x,y,w,h)=faces
#            img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
#            roi = img[y:y+h, x:x+w]
#
#            prediction = (model.predict([prepare(roi)]))
#            preds = prediction[0]
#            label = EMOTIONS[preds.argmax()]
#            emotion_count[preds.argmax()]+=1
#        except:
#            pass
#
#    cap.release()
#    cv2.destroyAllWindows()
#    
#    emo = emotion[str(np.argmax(emotion_count))]
#    
#    return emo

### Afraid: Hip Hop
### Angry: Rock, Metal
### Disgust: Hip Hop, Jazz
### Happy: Pop, Disco
### Neutral: Reggae, Classical
### Sad: Blues, Country, Classical
### Surprise: Disco


In [10]:
def egmap(emotionout):
    
    afraidlist = ["hiphop"]
    angrylist = ["rock", "metal"]
    disgustlist = ["hiphop", "jazz"]
    happylist = ["pop", "disco"]
    neutrallist = ["reggae", "classical"]
    sadlist = ["blues", "classical", "country"]
    surprisedlist = ["disco"]
    
    if emotionout is 'Afraid':
        genrechosen = random.choice(afraidlist)
    if emotionout is 'Angry':
        genrechosen = random.choice(angrylist)
    if emotionout is 'Disgust':
        genrechosen = random.choice(disgustlist)
    if emotionout is 'Happy':
        genrechosen = random.choice(happylist)
    if emotionout is 'Neutral':
        genrechosen = random.choice(neutrallist)
    if emotionout is 'Sad':
        genrechosen = random.choice(sadlist)
    if emotionout is 'Surprise':
        genrechosen = random.choice(surprisedlist)
    
    dirt = "Genre/"+genrechosen
    songchosen = random.choice(os.listdir(dirt))
    diro = "Genre/"+genrechosen+"/"+songchosen        
        
    return diro

In [11]:
def recommend(emotionout):
    print("Here's a song for you :")
    song = egmap(emotionout)

    audiofile = eyed3.load(song)

    Artist = audiofile.tag.artist
    Album = audiofile.tag.album
    Title = audiofile.tag.title
    
    print(Title+" by "+Artist)
    display(Audio(song, autoplay=True, rate=8000))

In [ ]:
#directory = "record.avi"
#emotionout = videoemotion()
#print("Your Recognised emotion is "+ emotionout)

In [ ]:
#recommend(emotionout)

In [12]:
def tunex():
    for i in range(0, 4):
        emotionout = videocap() 
        print("Your Recognised emotion is "+ emotionout)
        recommend(emotionout)
        time.sleep(20)

# TUNEX
### Tunes for Expressions
> by team AI/ML - DRISHTI
###### Maintain an emotion for 10 seconds and wait for the music to play.

In [ ]:
tunex()